In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.diagnostic import het_breuschpagan, het_white

from linearmodels.iv import IV2SLS
import numpy as np

In [2]:
# Import Data

df = pd.read_excel('Data6.xlsx')
sectors = pd.read_excel('Sectors.xlsx')
df.head()

,Eikon_tick,Factset_tick,Eikon_name,Factset_name,Comb_Score,Env_Score,Soc_Score,Gov_Score,Country,Underpricing,...,Ex_Turnover,Ex_Return,Firm_Commitment,ESG_Plus1,GICS_sector,GICS_industry,Year,ROA,Continent,Underwriter
0,TRVG.OQ,TRVG-USA,Trivago NV,trivago N.V. Sponsored ADR Class A,14.355586,0.000000,25.968447,8.629630,UNITED STATES,4.386364,...,2.270373e+08,0.055587,1,0,Communication Services,Media & Entertainment,2016,-0.059381,North America,J.P. Morgan
1,BEDU.N,BEDU-USA,Bright Scholar Education Holdings Ltd,Bright Scholar Education Holdings Ltd. Sponsor...,13.523392,0.000000,38.514093,2.162417,UNITED STATES,4.104762,...,6.236081e+07,0.006193,1,0,Consumer Discretionary,Consumer Services,2017,0.085107,North America,Morgan Stanley
2,CMCM.N,CMCM-USA,Cheetah Mobile Inc,"Cheetah Mobile, Inc. ADR Class A",24.858685,2.189055,25.556710,31.058496,UNITED STATES,4.035714,...,1.184004e+08,0.043745,1,0,Information Technology,Software & Services,2014,0.034784,North America,Morgan Stanley
3,TCTM.OQ,TCTM-USA,TCTM Kids IT Education Inc,TCTM Kids IT Education Inc. Sponsored ADR Class A,24.049320,0.000000,32.290471,33.947120,UNITED STATES,4.033334,...,1.220553e+08,0.031343,1,0,Consumer Discretionary,Consumer Services,2014,0.169980,North America,Goldman Sachs
4,JILL.N,JILL-USA,JJill Inc,"J.Jill, Inc.",33.704863,8.968927,50.858012,27.115558,UNITED STATES,3.865385,...,1.257601e+08,0.046620,1,0,Consumer Discretionary,Consumer Discretionary Distribution & Retail,2017,0.093801,North America,Merrill Lynch


In [3]:
# Avoid mechenical correlation by dropping overlapping entries in sector data

df['Eikon_tick'] = df['Eikon_tick'].astype(str).str.strip()
sectors['Identifier'] = sectors['Identifier'].astype(str).str.strip()

ipo_tickers = set(df['Eikon_tick'].unique())

overlapping_tickers = sectors[sectors['Identifier'].isin(ipo_tickers)].copy()

sectors_clean = sectors[~sectors['Identifier'].isin(ipo_tickers)].copy()

print(f"Dropped {len(overlapping_tickers)} overlapping rows from 'sectors'.")
if not overlapping_tickers.empty:
    print("The following tickers were removed from 'sectors':")
    print(overlapping_tickers['Identifier'].unique())
else:
    print("No overlapping tickers found between 'df' and 'sectors'.")


Dropped 1568 overlapping rows from 'sectors'.
The following tickers were removed from 'sectors':
['TMA.CS' 'EQUJ.J' 'DOMT.CA' ... 'VSL.AX' 'VNT.AX' 'LLL.AX']


In [4]:
# Create the dataframe with sector-level averages

years = list(range(2013, 2025))
relevant_columns = ['GICS_sector'] + years

sectors_subset = sectors[relevant_columns].copy()

sectors_avg = sectors_subset.groupby('GICS_sector')[years].mean().reset_index()

pd.set_option('display.precision', 3)
display(sectors_avg)

,GICS_sector,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Communication Services,40.141,39.247,40.234,40.478,40.326,39.508,40.030,40.556,41.192,41.297,42.575,41.595
1,Consumer Discretionary,38.656,38.244,39.665,40.661,40.949,41.675,42.856,42.999,43.473,44.429,45.264,44.616
2,Consumer Staples,43.170,43.023,44.735,44.417,44.639,44.265,45.071,44.709,45.190,46.286,47.205,46.429
3,Energy,40.338,39.468,39.728,40.029,38.639,40.766,41.464,42.375,43.537,43.898,44.571,43.228
4,Financials,41.071,40.838,41.710,41.791,41.533,41.877,42.490,42.958,44.089,44.575,45.466,45.283
5,Health Care,39.323,39.435,40.434,41.044,38.958,37.383,36.329,37.006,37.870,39.272,40.425,39.564
6,Industrials,41.406,41.610,41.667,42.394,42.199,42.568,43.429,44.506,45.114,45.096,45.931,46.012
7,Information Technology,44.503,44.109,43.957,45.431,43.880,42.790,42.998,42.756,42.997,44.047,44.511,44.447
8,Materials,41.261,40.713,42.288,43.431,42.626,43.387,44.030,44.177,45.301,45.084,46.312,46.282
9,Real Estate,39.503,39.534,39.654,39.901,40.838,42.467,44.257,46.027,47.232,47.934,48.427,48.939


In [5]:
# Check size of groups in sector data

years = list(range(2013, 2025))
relevant_columns = ['GICS_sector'] + years

sectors_subset = sectors[relevant_columns].copy()

sectors_long = sectors_subset.melt(id_vars='GICS_sector', value_vars=years, 
                                   var_name='Year', value_name='ESG_score')

sectors_long = sectors_long.dropna(subset=['ESG_score'])

sectors_count = sectors_long.groupby(['GICS_sector', 'Year'])['ESG_score'].count().reset_index(name='Count')

sectors_count_wide = sectors_count.pivot(index='GICS_sector', columns='Year', values='Count').reset_index()

pd.set_option('display.precision', 3)
display(sectors_count_wide)

Year,GICS_sector,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Communication Services,180,190,203,229,255,302,337,389,459,512,553,576
1,Consumer Discretionary,369,390,415,500,570,646,763,891,1040,1234,1335,1378
2,Consumer Staples,225,233,245,281,314,362,414,491,580,681,733,751
3,Energy,198,207,214,233,261,295,331,356,391,442,468,485
4,Financials,505,517,554,675,822,987,1120,1246,1376,1602,1683,1712
5,Health Care,169,182,207,264,354,520,682,834,1020,1178,1273,1323
6,Industrials,571,585,620,730,840,965,1131,1292,1526,1808,1978,2037
7,Information Technology,244,249,261,316,395,491,567,704,896,1041,1184,1220
8,Materials,389,397,407,439,482,537,615,713,835,1038,1161,1196
9,Real Estate,195,204,230,293,335,393,447,528,604,698,762,780


In [6]:
# Merge IPO data with sector data

sectors_avg.columns = ['GICS_sector'] + [int(c) for c in sectors_avg.columns if c != 'GICS_sector']

lookup = sectors_avg.set_index('GICS_sector')

def get_lagged_esg(row):
    sector = row['GICS_sector']
    lagged_year = row['Year'] - 1
    try:
        return lookup.at[sector, lagged_year]
    except KeyError:
        return float('nan') 

df['Sector_ESG'] = df.apply(get_lagged_esg, axis=1)

In [7]:
# Verify whether the merge succeeded 

n_missing = df['Sector_ESG'].isna().sum()
print(f"Missing Sector_ESG entries: {n_missing}")

pct_missing = df['Sector_ESG'].isna().mean() * 100
print(f"Share missing: {pct_missing:.2f}%")


Missing Sector_ESG entries: 0
Share missing: 0.00%


In [8]:
# create top tier underwriter dummy

underwriter_totals = df.groupby('Underwriter')['Deal_Size'].sum()

underwriter_ranked = underwriter_totals.sort_values(ascending=False)

top_10_underwriters = underwriter_ranked.head(10)
print(top_10_underwriters)

top_10_names = top_10_underwriters.index.tolist()

df['Top_Tier'] = df['Underwriter'].isin(top_10_names).astype(int)

Underwriter
Goldman Sachs                       1.493e+11
Morgan Stanley                      6.540e+10
J.P. Morgan                         6.277e+10
China International Capital Corp    5.928e+10
Nomura Securities Co., Ltd.         4.930e+10
ABCI                                4.043e+10
Deutsche Bank                       3.790e+10
Citigroup                           3.654e+10
Credit Suisse                       3.526e+10
BofA Securities                     2.928e+10
Name: Deal_Size, dtype: float64


In [9]:
# Final Cleaning

relevant = [
    'Underpricing',
    'Comb_Score',
    'Env_Score',
    'Soc_Score', 
    'Gov_Score',
    'Sector_ESG',
    'ROA', 
    'Deal_Size',
    'Ex_Turnover',
    'Ex_Return',
    'ESG_Plus1',
    'Firm_Commitment',
    'Top_Tier'
]

df_clean = df.dropna(subset=relevant).copy()
print(f"Dropped {len(df) - len(df_clean)} rows; {len(df_clean)} remain.")

df_clean['Deal_Size'] = np.log(df_clean['Deal_Size'].replace(0, np.nan))
df_clean['Ex_Turnover'] = np.log(df_clean['Ex_Turnover'].replace(0, np.nan))


esg_cols = ['Env_Score', 'Soc_Score', 'Gov_Score', 'Comb_Score', 'Sector_ESG']
scaler = StandardScaler()
df_clean[[col + '_std' for col in esg_cols]] = scaler.fit_transform(df_clean[esg_cols])


Dropped 109 rows; 1637 remain.


In [10]:
# Create Continent Dummies

continents = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

reference_region = 'Europe'

esg_vars = ['Comb_Score_std', 'Env_Score_std', 'Soc_Score_std', 'Gov_Score_std']

for continent in continents:
    if continent != reference_region:
        cname = continent.replace(" ", "")
        
        dummy_col = f'D_{cname}'
        df_clean[dummy_col] = (df_clean['Continent'] == continent).astype(int)
        
        for esg_var in esg_vars:
            interaction_col = f'{esg_var}_x_{cname}'
            df_clean[interaction_col] = df_clean[esg_var] * df_clean[dummy_col]



In [11]:
# Table 2: Descriptive Statistics

desc = df_clean[relevant].describe().T

desc['N'] = df_clean[relevant].count().astype(int)

desc = desc[['mean', 'std', 'min', 'max', 'N']].round(2)

desc.reset_index(inplace=True)
desc.rename(columns={'index': 'Variable'}, inplace=True)

desc['Variable'] = desc['Variable'].replace({
    'Underpricing': 'Underpricing',
    'Comb_Score': 'Comb\\_Score',
    'Env_Score': 'Env\\_Score',
    'Soc_Score': 'Soc\\_Score',
    'Gov_Score': 'Gov\\_Score',
    'ESG_Plus1': 'ESG\\_Plus1',
    'Sector_ESG': 'Sector\\_ESG',
    'Env_Score': 'Env\\_Score',
    'Deal_Size': 'Deal\\_Size',
    'Ex_Turnover': 'Ex\\_Turnover',
    'Ex_Return': 'Ex\\_Return',
    'Firm_Commitment': 'Firm\\_Commitment',
    'ROA': 'ROA',
    'Top_Tier': 'Top\\_Tier'
})

latex_table = desc.to_latex(index=False,
                            caption="Descriptive Statistics",
                            label="tab:descstats",
                            column_format='lccccc',
                            escape=False,
                            float_format="%.2f")

with open("descriptive_statistics.tex", "w") as f:
    f.write(latex_table)


In [12]:
# Appendix 1: Sector summary statistics

summary = df_clean.groupby("Country").agg(
    N=("Underpricing", "count"),
    Underpricing=("Underpricing", "mean"),
    Environmental=("Env_Score", "mean"),
    Social=("Soc_Score", "mean"),
    Governance=("Gov_Score", "mean"),
    Combined=("Comb_Score", "mean")
).reset_index()

summary["Underpricing"] *= 100

summary["Country"] = summary["Country"].str.title()

summary.rename(columns={"Underpricing": "Underpricing (\\%)"}, inplace=True)

summary = summary.sort_values("Country")

summary.to_latex(
    "country_summary_table.tex",
    index=False,
    column_format="lrrrrrr",
    escape=False,
    float_format="%.2f"
)


In [13]:
# Table 3: First Stage Regressions

pillars = ["Env_Score_std", "Soc_Score_std", "Gov_Score_std", "Comb_Score_std"]
fs_models_clustered = {}

for score in pillars:
    formula_fs = (
        f"{score} ~ 1 + Deal_Size + Ex_Turnover + Ex_Return + "
        "Firm_Commitment + ESG_Plus1 + ROA + Top_Tier + C(Country) + C(GICS_sector) + Sector_ESG_std"
    )
    fs_models_clustered[score] = ols(formula=formula_fs, data=df_clean).fit(
        cov_type="cluster", cov_kwds={"groups": df_clean["Country"]}
    )

def bp_stat(m):
    lm, lm_p, _, _ = het_breuschpagan(m.resid, m.model.exog)
    return lm, lm_p

def white_stat(m):
    stat, p, _, _ = het_white(m.resid, m.model.exog)
    return stat, p

info_dict = {
    "Sector_ESG Coef.": lambda m: f"{m.params['Sector_ESG_std']:.4f}",
    "Clustered SE":     lambda m: f"{m.bse['Sector_ESG_std']:.4f}",
    "P-value":          lambda m: f"{m.pvalues['Sector_ESG_std']:.4f}",
    "F-stat":           lambda m: f"{m.fvalue:.4f}",
    "BP p-value":       lambda m: f"{bp_stat(m)[1]:.4f}",
    "White p-value":    lambda m: f"{white_stat(m)[1]:.4f}"
}

table_clustered = summary_col(
    list(fs_models_clustered.values()),
    regressor_order=["Sector_ESG_std"],
    info_dict=info_dict,
    float_format="%0.4f",
    stars=True
)

with open("first_stage_sector_esg_clustered.tex", "w") as f:
    f.write(table_clustered.as_latex())

print("LaTeX table with country-clustered SEs and BP/White tests written to first_stage_sector_esg_clustered.tex")


C:\Users\Gijs\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 60, but rank is 18
  warnings.warn('covariance of constraints does not have full '
C:\Users\Gijs\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 60, but rank is 18
  warnings.warn('covariance of constraints does not have full '
C:\Users\Gijs\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 60, but rank is 18
  warnings.warn('covariance of constraints does not have full '
C:\Users\Gijs\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 60, but rank is 18
  warnings.warn('covariance of constraints does not have ful

LaTeX table with country-clustered SEs and BP/White tests written to first_stage_sector_esg_clustered.tex


In [14]:
#Table 4: Reduced Form 

controls = [
    "Deal_Size", "Ex_Turnover", "Ex_Return", 
    "Firm_Commitment", "ESG_Plus1", "ROA", "Top_Tier"
]
all_vars = ["Intercept", "Sector_ESG_std"] + controls

formula_rf = (
    f"Underpricing ~ 1 + {' + '.join(controls)} + C(Country) + C(GICS_sector) + Sector_ESG_std"
)
rf_model = ols(formula=formula_rf, data=df_clean).fit(
    cov_type="cluster", cov_kwds={"groups": df_clean["Country"]}
)

bp_lm, bp_p, _, _ = het_breuschpagan(rf_model.resid, rf_model.model.exog)
w_stat, w_p, _, _ = het_white(rf_model.resid, rf_model.model.exog)

rows = []
for var in all_vars:
    coef_row, se_row = [], []
    matches = [k for k in rf_model.params.index if var in k]
    if matches:
        k = matches[0]
        coef = rf_model.params[k]
        se = rf_model.bse[k]
        tval = rf_model.tvalues[k]
        if abs(tval) >= 2.576:
            stars = "***"
        elif abs(tval) >= 1.96:
            stars = "**"
        elif abs(tval) >= 1.645:
            stars = "*"
        else:
            stars = ""
        coef_row.append(f"{coef:.4f}{stars}")
        se_row.append(f"({se:.4f})")
    else:
        coef_row.append("")
        se_row.append("")
    rows.append((var, coef_row))
    rows.append(("", se_row))

def stat_row(name, value):
    return (name, [value])

rows.extend([
    stat_row("R-squared",      f"{rf_model.rsquared:.4f}"),
    stat_row("Adj. R2",        f"{rf_model.rsquared_adj:.4f}"),
    stat_row("F-stat",         f"{rf_model.fvalue:.4f}"),
    stat_row("N",              f"{int(rf_model.nobs)}"),
    stat_row("Country FE",     "Yes"),
    stat_row("Sector FE",      "Yes"),
    stat_row("Clustered SEs",  "Yes (Country)"),
    stat_row("BP p-value",     f"{bp_p:.4f}"),
    stat_row("White p-value",  f"{w_p:.4f}")
])

table_df = pd.DataFrame(
    [r[1] for r in rows],
    index=[r[0] for r in rows],
    columns=["Reduced Form"]
)

with open("reduced_form_table.tex", "w") as f:
    f.write(
        table_df.to_latex(
            na_rep="",        
            escape=False,     
            column_format="l c"
        )
    )

print("Wrote LaTeX table with country-clustered SEs to reduced_form_table.tex")


Wrote LaTeX table with country-clustered SEs to reduced_form_table.tex


C:\Users\Gijs\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 60, but rank is 18
  warnings.warn('covariance of constraints does not have full '


In [15]:
#Table 5: OLS with Country FE

pillars = ["Env_Score_std", "Soc_Score_std", "Gov_Score_std", "Comb_Score_std"]
controls = [
    "Deal_Size", "Ex_Turnover", "Ex_Return", "Firm_Commitment",
    "ESG_Plus1", "ROA", "Top_Tier"
]
all_vars = ["Intercept"] + pillars + controls

ols_models = {}
for pillar in pillars:
    formula = f"Underpricing ~ 1 + {' + '.join(controls)} + C(GICS_sector) + C(Country) + {pillar}"
    ols_models[pillar] = smf.ols(formula=formula, data=df_clean).fit(cov_kwds={'groups': df_clean['Country']})

rows = []
for var in all_vars:
    coef_row = []
    se_row = []
    for pillar in pillars:
        res = ols_models[pillar]
        matches = [k for k in res.params.index if var in k]
        if not matches:
            coef_row.append("")
            se_row.append("")
        else:
            k = matches[0]
            coef = res.params[k]
            se = res.bse[k]
            tval = res.tvalues[k]
            # stars
            if abs(tval) >= 2.576:
                stars = "***"
            elif abs(tval) >= 1.96:
                stars = "**"
            elif abs(tval) >= 1.645:
                stars = "*"
            else:
                stars = ""
            coef_row.append(f"{coef:>7.4f}{stars}")
            se_row.append(f"({se:>6.4f})")
    rows.append((var, coef_row))
    rows.append(("", se_row))

def stat_row(name, func):
    return (name, [func(ols_models[p]) for p in pillars])

bp_stats = {p: het_breuschpagan(m.resid, m.model.exog)[0] for p, m in ols_models.items()}
bp_pvals = {p: het_breuschpagan(m.resid, m.model.exog)[1] for p, m in ols_models.items()}
w_stats = {p: het_white(m.resid, m.model.exog)[0] for p, m in ols_models.items()}
w_pvals = {p: het_white(m.resid, m.model.exog)[1] for p, m in ols_models.items()}

rows.append(("", [""] * 4))  # spacer row
rows.append(("Sector FE", ["Yes"] * 4))
rows.append(("Country FE", ["Yes"] * 4))
rows.append(stat_row("N", lambda m: f"{int(m.nobs):>7}"))
rows.append(stat_row("R-squared", lambda m: f"{m.rsquared:>7.4f}"))
rows.append(stat_row("Adj. R2", lambda m: f"{m.rsquared_adj:>7.4f}"))
rows.append(stat_row("F-stat", lambda m: f"{m.fvalue:>7.2f}"))
rows.append(("BP p-value", [f"{bp_pvals[p]:>7.4f}" for p in pillars]))
rows.append(("White p-value", [f"{w_pvals[p]:>7.4f}" for p in pillars]))

table_df = pd.DataFrame(
    [r[1] for r in rows],
    index=[r[0] for r in rows],
    columns=pillars
)

with open("baseline_OLS", "w") as f:
    f.write(
        table_df.to_latex(
            na_rep="",
            escape=False,
            column_format="lcccc"
        )
    )

print("Wrote LaTeX table with BP and White tests to baseline_OLS.tex")


Wrote LaTeX table with BP and White tests to ols_all_pillars_with_tests.tex


In [16]:
# Table 6: 2SLS with Country FE

pillars = ["Env_Score_std", "Soc_Score_std", "Gov_Score_std", "Comb_Score_std"]
controls = [
    "Deal_Size", "Ex_Turnover", "Ex_Return", "Firm_Commitment",
    "ESG_Plus1", "ROA", "Top_Tier"
]
all_vars = ["Intercept"] + pillars + controls

iv_models     = {}
hausman_pvals = []
bp_pvals      = []
white_pvals   = []

for pillar in pillars:
    formula = (
        f"Underpricing ~ 1 + {' + '.join(controls)} + C(GICS_sector) + C(Country) "
        f"+ [{pillar} ~ Sector_ESG_std]"
    )
    iv_result = IV2SLS.from_formula(formula, data=df_clean).fit(
        cov_type="clustered", clusters=df_clean["Country"]
    )
    iv_models[pillar] = iv_result

    first_stage = smf.ols(
        formula=f"{pillar} ~ Sector_ESG_std + {' + '.join(controls)} + C(GICS_sector)+ C(Country)",
        data=df_clean
    ).fit()
    df_clean["v_hat"] = first_stage.resid
    hausman_model = smf.ols(
        formula=f"Underpricing ~ {pillar} + v_hat + {' + '.join(controls)} + C(GICS_sector)+ C(Country)",
        data=df_clean
    ).fit(cov_type="cluster", cov_kwds={"groups": df_clean["Country"]})
    hausman_pvals.append(hausman_model.pvalues.get("v_hat", float("nan")))

    resid = iv_result.resids

    sector_dummies = pd.get_dummies(
        df_clean["GICS_sector"], prefix="sector", drop_first=True
    )
    X = pd.concat([df_clean[controls], sector_dummies], axis=1)
    X = sm.add_constant(X)        # add intercept
    X = X.loc[resid.index]        # align with residuals

    bp_stat, bp_pval, _, _ = het_breuschpagan(resid, X)
    _, white_pval, _, _  = het_white(resid, X)

    bp_pvals.append(bp_pval)
    white_pvals.append(white_pval)

rows = []
for var in all_vars:
    coef_row, se_row = [], []
    for pillar in pillars:
        res = iv_models[pillar]
        candidates = [k for k in res.params.index if var in k]
        if not candidates:
            coef_row.append("")
            se_row.append("")
        else:
            k = candidates[0]
            coef = res.params[k]
            se   = res.std_errors[k]
            tval = res.tstats[k]
            if abs(tval) >= 2.576:
                stars = "***"
            elif abs(tval) >= 1.96:
                stars = "**"
            elif abs(tval) >= 1.645:
                stars = "*"
            else:
                stars = ""
            coef_row.append(f"{coef:.4f}{stars}")
            se_row.append(f"({se:.4f})")
    rows.append((var,   coef_row))
    rows.append(("",    se_row))

def stat_row(name, getter):
    return (name, [getter(iv_models[p]) for p in pillars])

rows.extend([
    ("Sector FE",         ["Yes"] * len(pillars)),
    ("Country FE",         ["Yes"] * len(pillars)),
    stat_row("N",          lambda m: f"{int(m.nobs)}"),
    stat_row("R-squared",  lambda m: f"{m.rsquared:.4f}"),
    stat_row("Adj. R2",    lambda m: f"{m.rsquared_adj:.4f}"),
    ("Durbin χ² p-value", [f"{p:.4f}" for p in hausman_pvals]),
    ("Breusch-Pagan p",    [f"{p:.4f}" for p in bp_pvals]),
    ("White p-value",      [f"{p:.4f}" for p in white_pvals]),
])

table_df = pd.DataFrame(
    [r[1] for r in rows],
    index=[r[0] for r in rows],
    columns=pillars
)

with open("2SLS_with_country_FE", "w", encoding="utf-8") as f:
    f.write(table_df.to_latex(
        na_rep="", escape=False, column_format="lcccc"
    ))

print("Wrote LaTeX table to: 2SLS_with_country_FE.tex")


Wrote LaTeX table to: second_stage_all_pillars.tex


In [17]:
#Table 9: OLS without Country FE

pillars = ["Env_Score_std", "Soc_Score_std", "Gov_Score_std", "Comb_Score_std"]
controls = [
    "Deal_Size", "Ex_Turnover", "Ex_Return", "Firm_Commitment",
    "ESG_Plus1", "ROA", "Top_Tier"
]
all_vars = ["Intercept"] + pillars + controls

ols_models = {}
for pillar in pillars:
    formula = f"Underpricing ~ 1 + {' + '.join(controls)} + C(GICS_sector) + {pillar}"
    ols_models[pillar] = smf.ols(formula=formula, data=df_clean).fit(cov_kwds={'groups': df_clean['Country']})

rows = []
for var in all_vars:
    coef_row = []
    se_row = []
    for pillar in pillars:
        res = ols_models[pillar]
        matches = [k for k in res.params.index if var in k]
        if not matches:
            coef_row.append("")
            se_row.append("")
        else:
            k = matches[0]
            coef = res.params[k]
            se = res.bse[k]
            tval = res.tvalues[k]
            # stars
            if abs(tval) >= 2.576:
                stars = "***"
            elif abs(tval) >= 1.96:
                stars = "**"
            elif abs(tval) >= 1.645:
                stars = "*"
            else:
                stars = ""
            coef_row.append(f"{coef:>7.4f}{stars}")
            se_row.append(f"({se:>6.4f})")
    rows.append((var, coef_row))
    rows.append(("", se_row))

def stat_row(name, func):
    return (name, [func(ols_models[p]) for p in pillars])

bp_stats = {p: het_breuschpagan(m.resid, m.model.exog)[0] for p, m in ols_models.items()}
bp_pvals = {p: het_breuschpagan(m.resid, m.model.exog)[1] for p, m in ols_models.items()}
w_stats = {p: het_white(m.resid, m.model.exog)[0] for p, m in ols_models.items()}
w_pvals = {p: het_white(m.resid, m.model.exog)[1] for p, m in ols_models.items()}

rows.append(("", [""] * 4))  # spacer row
rows.append(("Sector FE", ["Yes"] * 4))
rows.append(("Country FE", ["No"] * 4))
rows.append(stat_row("N", lambda m: f"{int(m.nobs):>7}"))
rows.append(stat_row("R-squared", lambda m: f"{m.rsquared:>7.4f}"))
rows.append(stat_row("Adj. R2", lambda m: f"{m.rsquared_adj:>7.4f}"))
rows.append(stat_row("F-stat", lambda m: f"{m.fvalue:>7.2f}"))
rows.append(("BP p-value", [f"{bp_pvals[p]:>7.4f}" for p in pillars]))
rows.append(("White p-value", [f"{w_pvals[p]:>7.4f}" for p in pillars]))

table_df = pd.DataFrame(
    [r[1] for r in rows],
    index=[r[0] for r in rows],
    columns=pillars
)

with open("OLS_without_country_FE", "w") as f:
    f.write(
        table_df.to_latex(
            na_rep="",
            escape=False,
            column_format="lcccc"
        )
    )

print("Wrote LaTeX table with BP and White tests to OLS_without_country_FE.tex")


Wrote LaTeX table with BP and White tests to ols_all_pillars_with_tests.tex


In [18]:
# Table 6: 2SLS without Country FE

pillars = ["Env_Score_std", "Soc_Score_std", "Gov_Score_std", "Comb_Score_std"]
controls = [
    "Deal_Size", "Ex_Turnover", "Ex_Return", "Firm_Commitment",
    "ESG_Plus1", "ROA", "Top_Tier"
]
all_vars = ["Intercept"] + pillars + controls

iv_models     = {}
hausman_pvals = []
bp_pvals      = []
white_pvals   = []

for pillar in pillars:
    formula = (
        f"Underpricing ~ 1 + {' + '.join(controls)} + C(GICS_sector)"
        f"+ [{pillar} ~ Sector_ESG_std]"
    )
    iv_result = IV2SLS.from_formula(formula, data=df_clean).fit(
        cov_type="clustered", clusters=df_clean["Country"]
    )
    iv_models[pillar] = iv_result

    first_stage = smf.ols(
        formula=f"{pillar} ~ Sector_ESG_std + {' + '.join(controls)} + C(GICS_sector)",
        data=df_clean
    ).fit()
    df_clean["v_hat"] = first_stage.resid
    hausman_model = smf.ols(
        formula=f"Underpricing ~ {pillar} + v_hat + {' + '.join(controls)} + C(GICS_sector)",
        data=df_clean
    ).fit(cov_type="cluster", cov_kwds={"groups": df_clean["Country"]})
    hausman_pvals.append(hausman_model.pvalues.get("v_hat", float("nan")))

    resid = iv_result.resids

    sector_dummies = pd.get_dummies(
        df_clean["GICS_sector"], prefix="sector", drop_first=True
    )
    X = pd.concat([df_clean[controls], sector_dummies], axis=1)
    X = sm.add_constant(X)        # add intercept
    X = X.loc[resid.index]        # align with residuals

    bp_stat, bp_pval, _, _ = het_breuschpagan(resid, X)
    _, white_pval, _, _  = het_white(resid, X)

    bp_pvals.append(bp_pval)
    white_pvals.append(white_pval)

rows = []
for var in all_vars:
    coef_row, se_row = [], []
    for pillar in pillars:
        res = iv_models[pillar]
        candidates = [k for k in res.params.index if var in k]
        if not candidates:
            coef_row.append("")
            se_row.append("")
        else:
            k = candidates[0]
            coef = res.params[k]
            se   = res.std_errors[k]
            tval = res.tstats[k]
            if abs(tval) >= 2.576:
                stars = "***"
            elif abs(tval) >= 1.96:
                stars = "**"
            elif abs(tval) >= 1.645:
                stars = "*"
            else:
                stars = ""
            coef_row.append(f"{coef:.4f}{stars}")
            se_row.append(f"({se:.4f})")
    rows.append((var,   coef_row))
    rows.append(("",    se_row))

def stat_row(name, getter):
    return (name, [getter(iv_models[p]) for p in pillars])

rows.extend([
    ("Sector FE",         ["Yes"] * len(pillars)),
    ("Country FE",         ["No"] * len(pillars)),
    stat_row("N",          lambda m: f"{int(m.nobs)}"),
    stat_row("R-squared",  lambda m: f"{m.rsquared:.4f}"),
    stat_row("Adj. R2",    lambda m: f"{m.rsquared_adj:.4f}"),
    ("Durbin χ² p-value", [f"{p:.4f}" for p in hausman_pvals]),
    ("Breusch-Pagan p",    [f"{p:.4f}" for p in bp_pvals]),
    ("White p-value",      [f"{p:.4f}" for p in white_pvals]),
])

table_df = pd.DataFrame(
    [r[1] for r in rows],
    index=[r[0] for r in rows],
    columns=pillars
)

with open("2SLS_without_country_FE", "w", encoding="utf-8") as f:
    f.write(table_df.to_latex(
        na_rep="", escape=False, column_format="lcccc"
    ))

print("Wrote LaTeX table to: 2SLS_without_country_FE.tex")


Wrote LaTeX table to: second_stage_all_pillars.tex


In [19]:
# OLS with dummies

pillars    = ["Env_Score_std", "Soc_Score_std", "Gov_Score_std", "Comb_Score_std"]
controls   = ["Deal_Size", "Ex_Turnover", "Ex_Return", "Firm_Commitment",
              "ESG_Plus1", "ROA", "Top_Tier"]
continents = ["Africa", "Asia", "NorthAmerica", "Oceania", "SouthAmerica"]

ols_models = {}
for p in pillars:
    dummies   = [f"D_{c}"    for c in continents]
    interacts = [f"{p}_x_{c}" for c in continents]
    formula = (
        "Underpricing ~ 1 + " +
        " + ".join(controls) +
        " + C(GICS_sector) + " +
        p + " + " +
        " + ".join(dummies) +
        " + " + " + ".join(interacts)
    )
    ols_models[p] = smf.ols(formula, data=df_clean) \
                     .fit(cov_kwds={"groups": df_clean["Country"]})

bp_pvals    = {p: het_breuschpagan(m.resid, m.model.exog)[1] for p, m in ols_models.items()}
white_pvals = {p: het_white(m.resid, m.model.exog)[1]    for p, m in ols_models.items()}

rows = []

coef_row = ["Intercept"]
se_row = [""]
for p in pillars:
    coef = ols_models[p].params.get("Intercept", np.nan)
    se   = ols_models[p].bse.get("Intercept", np.nan)
    t    = ols_models[p].tvalues.get("Intercept", 0.0)
    stars = "***" if abs(t) >= 2.576 else "**" if abs(t) >= 1.96 else "*" if abs(t) >= 1.645 else ""
    coef_row.append(f"{coef:.4f}{stars}")
    se_row.append(f"({se:.4f})")
rows.append(coef_row)
rows.append(se_row)


for ctrl in controls:
    coef_row = [ctrl] + [f"{ols_models[p].params[ctrl]: .4f}" for p in pillars]
    rows.append(coef_row)
    se_row   = [""]   + [f"({ols_models[p].bse[ctrl]: .4f})" for p in pillars]
    rows.append(se_row)

for kind in ("coef", "se"):
    if kind=="coef":
        label = "ESG Score"
        row   = [label]
        for p in pillars:
            coef = ols_models[p].params[p]
            t    = ols_models[p].tvalues[p]
            stars = "***" if abs(t)>=2.576 else "**" if abs(t)>=1.96 else "*" if abs(t)>=1.645 else ""
            row.append(f"{coef: .4f}{stars}")
    else:
        row = ["" ] + [f"({ols_models[p].bse[p]: .4f})" for p in pillars]
    rows.append(row)

for c in continents:
    param = lambda p: f"{p}_x_{c}"
    # coef row
    coef_row = [f"ESG Score × {c}"]
    for p in pillars:
        coef = ols_models[p].params.get(param(p), np.nan)
        t    = ols_models[p].tvalues.get(param(p), 0.0)
        stars = "***" if abs(t)>=2.576 else "**" if abs(t)>=1.96 else "*" if abs(t)>=1.645 else ""
        coef_row.append(f"{coef: .4f}{stars}")
    rows.append(coef_row)
    # SE row
    se_row = [""]
    for p in pillars:
        se = ols_models[p].bse.get(param(p), np.nan)
        se_row.append(f"({se: .4f})")
    rows.append(se_row)

for name, func in [
    ("N",          lambda m: int(m.nobs)),
    ("R-squared",  lambda m: f"{m.rsquared:.4f}"),
    ("Adj. R2",    lambda m: f"{m.rsquared_adj:.4f}"),
    ("F-stat",     lambda m: f"{m.fvalue:.2f}")
]:
    diag = [name] + [func(ols_models[p]) for p in pillars]
    rows.append(diag)

rows.append(["BP p-value"] + [f"{bp_pvals[p]:.4f}"    for p in pillars])
rows.append(["White p-value"] + [f"{white_pvals[p]:.4f}" for p in pillars])

table_df = pd.DataFrame(rows, columns=["Variable"] + pillars)
with open("OLS_withDummies.tex","w") as f:
    f.write(table_df.to_latex(
        index=False,
        na_rep="",
        escape=False,
        column_format="l" + "c"*len(pillars)
    ))

print("→ Wrote condensed table to OLS_withDummies.tex")


→ Wrote condensed table to OLS_Condensed_WithControls.tex


In [20]:
# 2SLS with dummies

from linearmodels.iv import IV2SLS

pillars = ["Env_Score_std", "Soc_Score_std", "Gov_Score_std", "Comb_Score_std"]
controls = [
    "Deal_Size", "Ex_Turnover", "Ex_Return", "Firm_Commitment",
    "ESG_Plus1", "ROA", "Top_Tier"
]
continents = ["Africa", "Asia", "NorthAmerica", "Oceania", "SouthAmerica"]

iv_models = {}
hausman_pvals, bp_pvals, white_pvals = [], [], []

for pillar in pillars:
    interactions = [f"{pillar}_x_{c}" for c in continents]
    exog_controls = controls + [f"D_{c}" for c in continents] + interactions

    formula = (
        f"Underpricing ~ 1 + {' + '.join(exog_controls)} + C(GICS_sector) "
        f"+ [{pillar} ~ Sector_ESG_std]"
    )

    model = IV2SLS.from_formula(formula, data=df_clean)
    res = model.fit(cov_type="clustered", clusters=df_clean["Country"])
    iv_models[pillar] = res

    first_stage = smf.ols(
        f"{pillar} ~ Sector_ESG_std + {' + '.join(exog_controls)} + C(GICS_sector)",
        data=df_clean
    ).fit()
    df_clean["v_hat"] = first_stage.resid
    hausman_test = smf.ols(
        f"Underpricing ~ {pillar} + v_hat + {' + '.join(controls)} + C(GICS_sector)",
        data=df_clean
    ).fit(cov_type="cluster", cov_kwds={"groups": df_clean["Country"]})
    hausman_pvals.append(hausman_test.pvalues.get("v_hat", float("nan")))

    resid = res.resids
    X = pd.concat([df_clean[controls], pd.get_dummies(df_clean["GICS_sector"], drop_first=True)], axis=1)
    X = sm.add_constant(X).loc[resid.index]

    bp_stat, bp_pval, _, _ = het_breuschpagan(resid, X)
    _, white_pval, _, _ = het_white(resid, X)
    bp_pvals.append(bp_pval)
    white_pvals.append(white_pval)

all_vars = (
    ["Intercept"]
    + controls
    + [f"D_{c}" for c in continents]
    + pillars
    + [f"{p}_x_{c}" for p in pillars for c in continents]
)

rows = []

coef_row = ["Intercept"]
se_row = [""]
for p in pillars:
    res = iv_models[p]
    coef = res.params.get("Intercept", np.nan)
    se = res.std_errors.get("Intercept", np.nan)
    t = res.tstats.get("Intercept", 0.0)
    stars = "***" if abs(t) >= 2.576 else "**" if abs(t) >= 1.96 else "*" if abs(t) >= 1.645 else ""
    coef_row.append(f"{coef:.4f}{stars}")
    se_row.append(f"({se:.4f})")
rows.append(coef_row)
rows.append(se_row)

for var in controls:
    coef_row = [var]
    se_row = [""]
    for p in pillars:
        res = iv_models[p]
        coef = res.params.get(var, np.nan)
        se = res.std_errors.get(var, np.nan)
        t = res.tstats.get(var, 0.0)
        stars = "***" if abs(t) >= 2.576 else "**" if abs(t) >= 1.96 else "*" if abs(t) >= 1.645 else ""
        coef_row.append(f"{coef:.4f}{stars}")
        se_row.append(f"({se:.4f})")
    rows.append(coef_row)
    rows.append(se_row)

coef_row = ["ESG Score"]
se_row = [""]
for p in pillars:
    res = iv_models[p]
    coef = res.params.get(p, np.nan)
    se = res.std_errors.get(p, np.nan)
    t = res.tstats.get(p, 0.0)
    stars = "***" if abs(t) >= 2.576 else "**" if abs(t) >= 1.96 else "*" if abs(t) >= 1.645 else ""
    coef_row.append(f"{coef:.4f}{stars}")
    se_row.append(f"({se:.4f})")
rows.append(coef_row)
rows.append(se_row)

for c in continents:
    label = f"ESG Score × {c}"
    coef_row = [label]
    se_row = [""]
    for p in pillars:
        param = f"{p}_x_{c}"
        res = iv_models[p]
        coef = res.params.get(param, np.nan)
        se = res.std_errors.get(param, np.nan)
        t = res.tstats.get(param, 0.0)
        stars = "***" if abs(t) >= 2.576 else "**" if abs(t) >= 1.96 else "*" if abs(t) >= 1.645 else ""
        coef_row.append(f"{coef:.4f}{stars}")
        se_row.append(f"({se:.4f})")
    rows.append(coef_row)
    rows.append(se_row)

rows.append(["Sector FE"] + ["Yes"] * len(pillars))
rows.append(["Country FE"] + ["No"] * len(pillars))
rows.append(["N"] + [f"{int(iv_models[p].nobs)}" for p in pillars])
rows.append(["R-squared"] + [f"{iv_models[p].rsquared:.4f}" for p in pillars])
rows.append(["Adj. R2"] + [f"{iv_models[p].rsquared_adj:.4f}" for p in pillars])
rows.append(["Durbin χ² p-value"] + [f"{p:.4f}" for p in hausman_pvals])
rows.append(["Breusch-Pagan p"] + [f"{p:.4f}" for p in bp_pvals])
rows.append(["White p-value"] + [f"{p:.4f}" for p in white_pvals])

table_df = pd.DataFrame(rows, columns=["Variable"] + pillars)
with open("2SLS_withDummies.tex", "w", encoding="utf-8") as f:
    f.write(table_df.to_latex(
        index=False,
        na_rep="",
        escape=False,
        column_format="l" + "c"*len(pillars)
    ))

print("→ Wrote condensed table to 2SLS_withDummies.tex")


→ Wrote condensed table to 2SLS_Condensed_WithControls.tex


In [21]:
# Appendix 2: Correlation Matrix 

vars_for_corr = [
    "Underpricing",
    "Env_Score", "Soc_Score", "Gov_Score", "Comb_Score", "Sector_ESG",
    "Deal_Size", "Ex_Turnover", "Ex_Return",
    "Firm_Commitment", "ESG_Plus1", "ROA", "Top_Tier"
]

df_corr = df_clean[vars_for_corr]
corr = df_corr.corr().round(4)
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
corr_tril = corr.mask(mask, "")

num = len(vars_for_corr)     
col_labels = [f"({i+1})" for i in range(num)]

corr_tril.insert(0, "Variable", vars_for_corr)

corr_tril.columns = ["Variable"] + col_labels

with open("corr_matrix_lower_triangle.tex", "w") as f:
    f.write(
        corr_tril.to_latex(
            index=False,
            header=True,
            na_rep="",
            float_format="%.2f",
            caption="Correlation matrix",
            label="tab:corr_matrix",
            column_format="l" + "r" * num
        )
    )